## モデルの能力を拡張する

大規模言語モデル (LLM) は非常に強力なツールですが、訓練された知識や情報の範囲内でしかその力を発揮できません。結局のところ、知っていることしか答えられないということです。しかし、もし訓練データに含まれていない質問をしたい場合はどうでしょうか？また、訓練データに含まれていないが、関連する質問をしたい場合は？

この問題を解決する方法はいくつかありますが、使えるリソースや、費やせる時間や費用に応じて異なります。以下はそのいくつかのオプションです。

- 必要な情報を含むようにモデルを完全に再訓練する。これは、大規模言語モデルの場合、数千のGPUを数週間にわたって稼働させることができる数社のみが可能です。
- この新しい情報を使ってモデルを微調整する。これにははるかに少ないリソースが必要で、通常は数時間または数分で完了します（モデルのサイズによります）。ただし、完全に再訓練するわけではないため、新しい情報が回答に完全に統合されない場合があります。微調整は特定のコンテキストや語彙をよりよく理解するのに優れており、新しい知識を注入する点では少し劣ります。また、新しい情報を追加するたびに再訓練と再デプロイが必要です。
- 新しい情報をデータベースに格納し、クエリに関連する部分をクエリのコンテキストとして取り出してから、LLMに送信する。この技術は**検索強化生成 (Retrieval Augmented Generation, RAG)** と呼ばれます。再訓練や微調整を必要とせず、いつでも簡単に更新可能な新しい知識を活用できる点で興味深い技術です。

すでに [Milvus](https://milvus.io/) を使用してベクターデータベースを準備し、[埋め込み](https://www.ibm.com/topics/embedding)形式で[カリフォルニア運転手ハンドブック](https://www.dmv.ca.gov/portal/handbook/california-driver-handbook/)の内容を保存しています。

このノートブックでは、RAG を使用して**ある請求に関するクエリ**を行い、この新しい知識がどのように役立つかを、モデルを変更せずに確認します。

### 必要なものとインポート

Lab の指示に従って適切なワークベンチイメージを選択した場合は、すでに必要なライブラリがすべて揃っているはずです。もしそうでない場合は、次のセルの最初の行のコメントを解除して、正しいパッケージをインストールしてください。

In [ ]:
# !pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt # 適切なワークベンチイメージを選択していない場合のみコメントを解除

import json
import os
from os import listdir
from os.path import isfile, join
from langchain.chains import LLMChain
from langchain.chains import LLMChain, RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.llms import VLLMOpenAI
from langchain_community.vectorstores import Milvus
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from milvus_retriever_with_score_threshold import MilvusRetrieverWithScoreThreshold

# 埋め込みモデルをダウンロードする際の警告をオフにする
import transformers
transformers.logging.set_verbosity_error()

### Langchain の要素

再び、Langchain を使ってタスクパイプラインを定義します。

まずは、クエリを送信するための **LLM** です。

In [ ]:
# LLM 推論サーバーの URL
inference_server_url = "http://llm.ic-shared-llm.svc.cluster.local:8000"

# LLM の定義
llm = VLLMOpenAI(           # vLLM OpenAI 互換の API クライアントを使用します。ただし、モデルは OpenShift 上で稼働しており、OpenAI 上ではありません。
    openai_api_key="EMPTY",   # そのため、OpenAI のキーは不要です。
    openai_api_base= f"{inference_server_url}/v1",
    model_name="mistralai/Mistral-7B-Instruct-v0.2",
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

次に、カリフォルニア運転手ハンドブックを準備し保存した **ベクターデータベース** への接続です。

In [ ]:
# まず、ハンドブックの処理に使用した埋め込みを定義します
model_kwargs = {"trust_remote_code": True}
embeddings = HuggingFaceEmbeddings(
            model_name="nomic-ai/nomic-embed-text-v1",
            model_kwargs=model_kwargs,
            show_progress=False,
        )

# 次に、Milvus ベクトルストアから関連データを取得するためのリトリーバーを定義します
retriever = MilvusRetrieverWithScoreThreshold(
            embedding_function=embeddings,
            collection_name="california_driver_handbook_1_0",
            collection_description="",
            collection_properties=None,
            connection_args={
                "host": "vectordb.milvus.svc.cluster.local",
                "port": 19530,
            },
            score_threshold=0.7, # 閾値を 0.7 に設定
            max_result_count=5,  # 最大 5 件の結果を返す
        )